# [Chapter 5. Files and I/O](http://chimera.labs.oreilly.com/books/1230000000393/ch05.html)

All programs need to perform input and output.  
This chapter covers common idioms for working with different kinds of files, including text and binary files, file encodings, and other related matters.  
Techniques for manipulating filenames and directories are also covered.

## [Reading and Writing Text Data](http://chimera.labs.oreilly.com/books/1230000000393/ch05.html#_reading_and_writing_text_data)

### Problem 

You need to read or write text data, possibly in different text encodings such as ASCII, UTF-8, or UTF-16.

### Solution

Use the `open()` function with mode `rt` to read a text file.